# Incrustaciones con BETO

La incrustación (*embedding*) es el proceso por el que obtenemos números a partir de palabras. En el cuadernon `5-Sentimiento.ipynb` ya he realizado la limpieza de la columna `Description`. Por tanto, usaré el dataframe que obtuve de ese proceso para obtener la tokenización con [BETO](https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased), la versión de BERT en español.

In [7]:
import pandas as pd
import numpy as np
import re 

Reviso las columnas que tengo en mi dataframe.

In [36]:
base=pd.read_csv('rawdata/clases/habitaclia5467_ejercicio3.csv')
print(base.columns.tolist())

['Unnamed: 0.1', 'Unnamed: 0', 'OBJECTID', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_em

La columna que había obtenido se llama `cl_descrip`. Así que sobre ella trabajo. 

**Cuidado**: la siguiente celda de código puede tomar entre 10-15 minutos para ejecutarse.

In [37]:
import torch
from transformers import BertTokenizer, BertModel

# Cargar el tokenizador y el modelo BERT preentrenado en español
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# Definir una función para generar las incrustaciones BERT
def generate_bert_embeddings(description):
    if not isinstance(description, str) or not description.strip():
        return torch.zeros(1, 768)  # Devolver un tensor de ceros para descripciones no válidas
    else:
        # Tokenizar la descripción
        tokens = tokenizer(description, return_tensors='pt', padding=True, truncation=True)
        
        # Obtener las incrustaciones BERT
        with torch.no_grad():
            outputs = model(**tokens)
            embeddings = outputs.last_hidden_state.mean(dim=1)  # Promedio de los embeddings de todas las palabras
        
        return embeddings.squeeze()

# Aplicar la función a la columna 'cl_descrip' y almacenar los resultados en una nueva columna 'BERT'
base['BERT'] = base['cl_descrip'].apply(lambda x: generate_bert_embeddings(x))

# Ver el DataFrame resultante
base['BERT'].head()

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0    [tensor(-0.0741), tensor(0.0608), tensor(-0.29...
1    [tensor(-0.1882), tensor(-0.3323), tensor(0.08...
2    [tensor(0.0010), tensor(0.3503), tensor(-0.295...
3    [tensor(-0.2751), tensor(-0.1341), tensor(0.07...
4    [tensor(-0.3513), tensor(-0.1383), tensor(0.15...
Name: BERT, dtype: object

Se han generado varios valores, pero por 768 dimensiones, que es el número de dimensiones con las que trabaja BETO. Ahora tenemos que hacer el mean-pooling. Le voy a indicar que aquellas filas sin descripción tengan únicamente valores 0.

In [38]:
def mean_pooling(row):
    # Convierte la columna 'BERT' en una matriz NumPy
    bert_array = np.array(row['BERT'])
    # Calcula la media a lo largo del eje 0
    return np.mean(bert_array, axis=0)

# Aplicamos la función para calcular la media de los vectores BERT
base['mpBERT'] = base.apply(mean_pooling, axis=1)

base['mpBERT'].head()

0   -0.028312
1   -0.028498
2    -0.02868
3   -0.028602
4   -0.028879
Name: mpBERT, dtype: object

Si quieren, pueden exportar a Excel lo que han obtenido para ir revisando un poco su estructura.

In [40]:
#base[['cl_descrip','BERT','mpBERT']].to_excel('rawdata/clases/habitaclia5467_ejercicio3-v2.xlsx')

He notado que he obtenido el mean-pooling de cada fila, con excepción de aquellas con valores 0. Voy a corregir eso. 

In [41]:
# Verificar si hay valores que no sean números en la columna 'Columna1'
valores_no_numericos = pd.to_numeric(base['mpBERT'], errors='coerce').isnull()

# Convertir los valores no numéricos a 0
base.loc[valores_no_numericos, 'mpBERT'] = 0

Ahora sí exporto.

In [42]:
base.to_csv('rawdata/clases/habitaclia5467_ejercicio3-v2.csv') 
#Antes lo hice en excel, porque es más sencillo así explorar.
#Ahora lo guardo en csv. Es cuestión de gustos.